In [18]:
"""Pivot table in pandas."""

'Pivot table in pandas.'

# Сводная таблица в pandas

*Сводная таблица* - это мощный инструмент для обобщения и представления данных. 

В Pandas есть функция [`DataFrame.pivot_table()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot_table.html), которая позволяет быстро преобразовать [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) в сводную таблицу.

Обобщенная схема работы функции `pivot_table`:

<img src="https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/pivot_table_pandas.png" >

Эта функция очень полезна, но иногда бывает сложно запомнить, как ее использовать для форматирования данных нужным вам способом.

В этом Блокноте рассказывается, как использовать `pivot_table`.

Полный текст оригинальной статьи находится [здесь](http://pbpython.com/pandas-pivot-table-explained.html).

В этом сценарии я собираюсь отслеживать воронку (план) продаж (также называемую воронкой, funnel). Основная проблема заключается в том, что некоторые циклы продаж очень длинные (например, "корпоративное программное обеспечение", капитальное оборудование и т.д.), и руководство хочет отслеживать их детально в течение года.

Типичные вопросы, относящиеся к таким данным, включают:

Какой доход находится в воронке (плане продаж)?
Какие продукты находятся в воронке?
У кого какие продукты на каком этапе?
Насколько вероятно, что мы закроем сделки к концу года?

In [19]:
import numpy as np
import pandas as pd

Прочтите данные о нашей воронке продаж в `DataFrame`:

In [20]:
# pylint: disable=line-too-long

df = pd.read_excel(
    "https://github.com/dm-fedorov/pandas_basic/raw/master/%D0%B1%D1%8B%D1%81%D1%82%D1%80%D0%BE%D0%B5%20%D0%B2%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B2%20pandas/data/salesfunnel.xlsx"
)
df.head()
# Счет, Название компании, Представитель компании, Менеджер по продажам, Продукт, Кол-во, Стоимость, Статус сделки

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


Для удобства давайте представим столбец `Status` как [категориальную переменную](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) (`category`) и установим порядок, в котором хотим просматривать.

Это не является строго обязательным, но помогает поддерживать желаемый порядок при работе с данными.

In [ ]:
df["Status"] = df["Status"].astype("category")
df["Status"] = df["Status"].cat.set_categories(
    ["Ordered", "Shipped", "Delivered", "Returned"]
)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Account   17 non-null     int64   
 1   Name      17 non-null     object  
 2   Rep       17 non-null     object  
 3   Manager   17 non-null     object  
 4   Product   17 non-null     object  
 5   Quantity  17 non-null     int64   
 6   Price     17 non-null     int64   
 7   Status    0 non-null      category
dtypes: category(1), int64(3), object(4)
memory usage: 1.3+ KB


# Поворот данных

Создавать сводную таблицу (`pivot table`) проще всего последовательно. Добавляйте элементы по одному и проверяйте каждый шаг, чтобы убедиться, что вы получаете ожидаемые результаты.

Самая простая сводная таблица должна иметь `DataFrame` и индекс (`index`). В этом примере давайте использовать `Name` в качестве индекса:

In [ ]:
numeric_cols = df.select_dtypes(include=["number"]).columns
pd.pivot_table(df, index=["Name"], values=numeric_cols)  # type: ignore[call-overload]

,Account,Price,Quantity
Name,,,
Barton LLC,740150.0,35000.0,1.000000
"Fritsch, Russel and Anderson",737550.0,35000.0,1.000000
Herman LLC,141962.0,65000.0,2.000000
Jerde-Hilpert,412290.0,5000.0,2.000000
"Kassulke, Ondricka and Metz",307599.0,7000.0,3.000000
Keeling LLC,688981.0,100000.0,5.000000
Kiehn-Spinka,146832.0,65000.0,2.000000
Koepp Ltd,729833.0,35000.0,2.000000
Kulas Inc,218895.0,25000.0,1.500000


У вас может быть несколько индексов. Фактически, большинство аргументов pivot_table могут принимать несколько значений в качестве элементов списка:

In [24]:
pd.pivot_table(
    df,
    index=["Name", "Rep", "Manager"],
    values=df.select_dtypes(include="number").columns.tolist(),
)

,,,Account,Price,Quantity
Name,Rep,Manager,,,
Barton LLC,John Smith,Debra Henley,740150.0,35000.0,1.000000
"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,737550.0,35000.0,1.000000
Herman LLC,Cedric Moss,Fred Anderson,141962.0,65000.0,2.000000
Jerde-Hilpert,John Smith,Debra Henley,412290.0,5000.0,2.000000
"Kassulke, Ondricka and Metz",Wendy Yule,Fred Anderson,307599.0,7000.0,3.000000
Keeling LLC,Wendy Yule,Fred Anderson,688981.0,100000.0,5.000000
Kiehn-Spinka,Daniel Hilton,Debra Henley,146832.0,65000.0,2.000000
Koepp Ltd,Wendy Yule,Fred Anderson,729833.0,35000.0,2.000000
Kulas Inc,Daniel Hilton,Debra Henley,218895.0,25000.0,1.500000


Это интересно, но не особо полезно. 

Мы хотим посмотреть на эти данные со стороны менеджера (`Manager`) и директора (`Director`). Это достаточно просто сделать, изменив индекс:

In [26]:
pd.pivot_table(
    df,
    index=["Manager", "Rep"],
    values=df.select_dtypes(include="number").columns.tolist(),
)

Account         Price  Quantity
Manager       Rep                                            
Debra Henley  Craig Booker   720237.0  20000.000000  1.250000
              Daniel Hilton  194874.0  38333.333333  1.666667
              John Smith     576220.0  20000.000000  1.500000
Fred Anderson Cedric Moss    196016.5  27500.000000  1.250000
              Wendy Yule     614061.5  44250.000000  3.000000

Вы могли заметить, что сводная таблица достаточно умна, чтобы начать агрегирование данных и их обобщение, группируя представителей (`Rep`) с их менеджерами (`Manager`). Теперь мы начинаем понимать, что может сделать для нас сводная таблица.

Давайте удалим счет (`Account`) и количество (`Quantity`), явно определив столбцы, которые нам нужны, с помощью параметра `values`:

In [27]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"])

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000

Столбец цен (`price`) по умолчанию усредняет данные, но мы можем произвести подсчет количества или суммы. Добавить их можно с помощью параметра `aggfunc`:

In [28]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], aggfunc=np.sum)

C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\4045244922.py:1: FutureWarning: The provided callable <function sum at 0x0000029D7A0B9D00> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], aggfunc=np.sum)


Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

`aggfunc` может принимать список функций. 

Давайте попробуем узнать среднее значение и количество:

In [29]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], aggfunc=[np.mean, len])

C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\2066971706.py:1: FutureWarning: The provided callable <function mean at 0x0000029D7A0BB100> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], aggfunc=[np.mean, len])


mean   len
                                    Price Price
Manager       Rep                              
Debra Henley  Craig Booker   20000.000000     4
              Daniel Hilton  38333.333333     3
              John Smith     20000.000000     2
Fred Anderson Cedric Moss    27500.000000     4
              Wendy Yule     44250.000000     4

Если мы хотим увидеть продажи с разбивкой по продуктам (`Product`), переменная `columns` позволяет нам определить один или несколько столбцов.

Я думаю, что одна из сложностей `pivot_table` - это использование столбцов (`columns`) и значений (`values`). 

Помните, что столбцы необязательны - они предоставляют дополнительный способ сегментировать актуальные значения, которые вам нужны. 

Функции агрегирования применяются к перечисленным значениям (`values`):

In [30]:
pd.pivot_table(
    df,
    index=["Manager", "Rep"],
    values=["Price"],
    columns=["Product"],
    aggfunc=[np.sum],
)

C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\4283755970.py:1: FutureWarning: The provided callable <function sum at 0x0000029D7A0B9D00> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(


sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

Значения `NaN` немного отвлекают. Если мы хотим их убрать, то можем использовать параметр `fill_value`, чтобы установить в `0`.

In [31]:
pd.pivot_table(
    df,
    index=["Manager", "Rep"],
    values=["Price"],
    columns=["Product"],
    aggfunc=[np.sum],
    fill_value=0,
)

C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\1964839742.py:1: FutureWarning: The provided callable <function sum at 0x0000029D7A0B9D00> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(


sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

Думаю, было бы полезно добавить количество (`Quantity`). 

Добавьте количество (`Quantity`) в список значений `values`:

In [32]:
pd.pivot_table(
    df,
    index=["Manager", "Rep"],
    values=["Price", "Quantity"],
    columns=["Product"],
    aggfunc=[np.sum],
    fill_value=0,
)

C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\228223386.py:1: FutureWarning: The provided callable <function sum at 0x0000029D7A0B9D00> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(


sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

Что интересно, вы можете добавлять элементы в индекс, чтобы получить другое визуальное представление. 

Добавим товары (`Products`) в индекс.

In [33]:
pd.pivot_table(
    df,
    index=["Manager", "Rep", "Product"],
    values=["Price", "Quantity"],
    aggfunc=[np.sum],
    fill_value=0,
)

C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\2950711180.py:1: FutureWarning: The provided callable <function sum at 0x0000029D7A0B9D00> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(


sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2

Для этого набора данных такое представление имеет больше смысла. 

А что, если я хочу увидеть некоторые итоги? `margins=True` делает это за нас.

In [34]:
pd.pivot_table(
    df,
    index=["Manager", "Rep", "Product"],
    values=["Price", "Quantity"],
    aggfunc=[np.sum, np.mean],
    fill_value=0,
    margins=True,
)

C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\3980171243.py:1: FutureWarning: The provided callable <function sum at 0x0000029D7A0B9D00> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(
C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\3980171243.py:1: FutureWarning: The provided callable <function mean at 0x0000029D7A0BB100> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pd.pivot_table(


sum                   mean  \
                                          Price Quantity         Price   
Manager       Rep           Product                                      
Debra Henley  Craig Booker  CPU           65000        2  32500.000000   
                            Maintenance    5000        2   5000.000000   
                            Software      10000        1  10000.000000   
              Daniel Hilton CPU          105000        4  52500.000000   
                            Software      10000        1  10000.000000   
              John Smith    CPU           35000        1  35000.000000   
                            Maintenance    5000        2   5000.000000   
Fred Anderson Cedric Moss   CPU           95000        3  47500.000000   
                            Maintenance    5000        1   5000.000000   
                            Software      10000        1  10000.000000   
              Wendy Yule    CPU          165000        7  82500.000000   
                            Maintenance    7000        3   7000.000000   
                            Monitor        5000        2   5000.000000   
All                                      522000       30  30705.882353   

                                                   
                                         Quantity  
Manager       Rep           Product                
Debra Henley  Craig Booker  CPU          1.000000  
                            Maintenance  2.000000  
                            Software     1.000000  
              Daniel Hilton CPU          2.000000  
                            Software     1.000000  
              John Smith    CPU          1.000000  
                            Maintenance  2.000000  
Fred Anderson Cedric Moss   CPU          1.500000  
                            Maintenance  1.000000  
                            Software     1.000000  
              Wendy Yule    CPU          3.500000  
                            Maintenance  3.000000  
                            Monitor      2.000000  
All                                      1.764706

Давайте переместим анализ на уровень выше и посмотрим на наш план продаж (воронку) на уровне менеджера.

Обратите внимание на то, как статус упорядочен на основе нашего предыдущего определения категории.

In [35]:
pd.pivot_table(
    df,
    index=["Manager", "Status"],
    values=["Price"],
    aggfunc=[np.sum],
    fill_value=0,
    margins=True,
)

C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\198624817.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pd.pivot_table(
C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\198624817.py:1: FutureWarning: The provided callable <function sum at 0x0000029D7A0B9D00> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(


sum
                        Price
Manager       Status         
Debra Henley  Ordered       0
              Shipped       0
              Delivered     0
              Returned      0
Fred Anderson Ordered       0
              Shipped       0
              Delivered     0
              Returned      0
All                         0

Очень удобно передать словарь в качестве `aggfunc`, чтобы вы могли выполнять разные функции с каждым из выбранных значений. Это имеет побочный эффект - названия становятся немного чище:

In [36]:
pd.pivot_table(
    df,
    index=["Manager", "Status"],
    columns=["Product"],
    values=["Quantity", "Price"],
    aggfunc={"Quantity": len, "Price": np.sum},
    fill_value=0,
)

C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\4079315380.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pd.pivot_table(
C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\4079315380.py:1: FutureWarning: The provided callable <function sum at 0x0000029D7A0B9D00> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(


Price                             
Product                   CPU Maintenance Monitor Software
Manager       Status                                      
Debra Henley  Ordered       0           0       0        0
              Shipped       0           0       0        0
              Delivered     0           0       0        0
              Returned      0           0       0        0
Fred Anderson Ordered       0           0       0        0
              Shipped       0           0       0        0
              Delivered     0           0       0        0
              Returned      0           0       0        0

Вы также можете предоставить список агрегированных функций (aggfunctions), которые будут применяться к каждому значению:

In [37]:
table = pd.pivot_table(
    df,
    index=["Manager", "Status"],
    columns=["Product"],
    values=["Quantity", "Price"],
    aggfunc={"Quantity": len, "Price": [np.sum, np.mean]},  # type: ignore
    fill_value=0,
)

C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\2326720483.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  table = pd.pivot_table(
C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\2326720483.py:1: FutureWarning: The provided callable <function sum at 0x0000029D7A0B9D00> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  table = pd.pivot_table(
C:\Users\Ruslan\AppData\Local\Temp\ipykernel_6528\2326720483.py:1: FutureWarning: The provided callable <function mean at 0x0000029D7A0BB100> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  table = pd.pivot_table(


table

Может показаться сложным собрать все это сразу, но как только вы начнете играть с данными и медленно добавлять элементы, то почувствуете, как это работает.

Мое общее практическое правило заключается в том, что после использования нескольких группировок (`grouby`) вы должны оценить, является ли сводная таблица (`pivot table`) полезным подходом.

# Расширенная фильтрация сводной таблицы

После того, как вы сгенерировали свои данные, они находятся в `DataFrame`, поэтому можно фильтровать их, используя обычные методы `DataFrame`.

Если вы хотите посмотреть только на одного менеджера:

In [38]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html

table.query('Manager == ["Debra Henley"]')

Price                                               \
                        mean                              sum               
Product                  CPU Maintenance Monitor Software CPU Maintenance   
Manager      Status                                                         
Debra Henley Ordered     0.0         0.0     0.0      0.0   0           0   
             Shipped     0.0         0.0     0.0      0.0   0           0   
             Delivered   0.0         0.0     0.0      0.0   0           0   
             Returned    0.0         0.0     0.0      0.0   0           0   

                                         
                                         
Product                Monitor Software  
Manager      Status                      
Debra Henley Ordered         0        0  
             Shipped         0        0  
             Delivered       0        0  
             Returned        0        0

Мы можем просмотреть все незавершенные (`pending`) и выигранные (`won`) сделки:

In [39]:
table.query('Status == ["pending", "won"]')

Empty DataFrame
Columns: [(Price, mean, CPU), (Price, mean, Maintenance), (Price, mean, Monitor), (Price, mean, Software), (Price, sum, CPU), (Price, sum, Maintenance), (Price, sum, Monitor), (Price, sum, Software)]
Index: []

Я надеюсь, что этот пример показал вам, как использовать сводные таблицы в собственных наборах данных.

# Шпаргалка

Схема с примером из Блокнота:

<img src="https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/pivot-table-datasheet.png" >